# Waze Partner Feed Data API

In [ ]:
import requests, json, os
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns; sns.set()
from matplotlib.path import Path as mpl_path
from datetime import datetime

C:\Users\luisr\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### Waze partner feed methods

In [36]:
waze_url = 'https://www.waze.com/partnerhub-api/waze-feed-access-token/c37c11ba-ff9d-4ad5-8ecc-4e4f12e91efb?format=1'

def get_waze_partner_alerts(alert_type='alerts'):
    "alert_type: one of 'alerts', 'irregularities', 'jams'"
    incidents = requests.get(waze_url).json()
    if alert_type not in incidents.keys():
        return None
    df = pd.DataFrame(incidents[alert_type])
    # Data cleaning & preprocessing
    if 'location' in df.columns:
        df[['latitude', 'longitude']] = list(df['location'].map(lambda coords: [coords['x'], coords['y']]))
        df.drop('location', axis=1, inplace=True)
    if 'pubMillis' in df.columns:
        df['pubMillis'] = (df['pubMillis'] / 1000).map(datetime.fromtimestamp)
        df.sort_values('pubMillis', ascending=False, inplace=True)
        df.reset_index(drop=True, inplace=True)
    return df

def get_waze_partner_alerts_extended():
    # Get waze alerts
    alerts = get_waze_partner_alerts(alert_type='alerts')
    # Get events points array
    points = np.array(list(map(tuple, alerts[['latitude', 'longitude']].values)))
    # Get clusters polygons events dict
    events_poly = {}
    for cluster_id, poly in polygons.items():
        mpl_poly =  mpl_path(poly)
        points_msk = mpl_poly.contains_points(points)
        poly_events_df = alerts[points_msk]
        poly_events_ids = list(poly_events_df['uuid'])
        for event_id in poly_events_ids:
            events_poly[event_id] = cluster_id
    
    # Update events dataframe with events polygons ids
    alerts['cluster_id'] = -1
    for event_id, cluster_id in events_poly.items():
        alerts['cluster_id'][alerts['uuid']==event_id] = cluster_id

    # Return extended open events
    return alerts

def get_clusters_waze_partner_alerts():
    # Get waze alerts
    alerts = get_waze_partner_alerts(alert_type='alerts')
    # Get events points array
    points = np.array(list(map(tuple, alerts[['latitude', 'longitude']].values)))
    # Get clusters polygons events dict
    poly_events = []
    for cluster_id, poly in polygons.items():
        mpl_poly =  mpl_path(poly)
        points_msk = mpl_poly.contains_points(points)
        poly_events_df = alerts[points_msk]
        poly_events_ids = list(poly_events_df['uuid'])
        poly_wb_events_ids = list(poly_events_df['uuid'][poly_events_df['subtype']=="HAZARD_WEATHER_FLOOD"])
        poly_events.append({
            'cluster_id': cluster_id,
            'alerts': len(poly_events_ids),
            'alerts_ids': poly_events_ids,
            'alerts_status': int(bool(len(poly_events_ids))),
            'waterbag_alerts': len(poly_wb_events_ids),
            'waterbag_alerts_ids': poly_wb_events_ids,
            'waterbag_alerts_status': int(bool(len(poly_wb_events_ids))),
        })
    return pd.DataFrame(poly_events)

---
## Waze partner feed request

In [12]:
alerts = get_waze_partner_alerts(alert_type='alerts')

alerts.head()

,country,city,reportRating,confidence,reliability,type,uuid,roadType,magvar,subtype,street,pubMillis,nThumbsUp,reportDescription,latitude,longitude
0,BR,NaN,0,0,5,ACCIDENT,302e57f6-73c6-4547-9fe5-41eaf69af136,6.0,73,ACCIDENT_MINOR,BR-116 S Rod. Pres. Dutra,2022-11-13 01:22:30,NaN,NaN,-43.822723,-22.656735
1,BR,Rio de Janeiro,2,0,6,JAM,7ef7f1ca-48e4-4ef4-be5d-347e91f1c878,2.0,0,JAM_STAND_STILL_TRAFFIC,Av. Ayrton Senna - Pista Central,2022-11-13 01:21:55,0.0,NaN,-43.365471,-22.980718
2,BR,Rio de Janeiro,3,0,5,WEATHERHAZARD,10a1abfa-955e-4200-812a-19268e1c864b,6.0,242,HAZARD_ON_SHOULDER_CAR_STOPPED,Av. Brasil,2022-11-13 01:21:55,NaN,NaN,-43.611198,-22.874937
3,BR,Rio de Janeiro,3,0,5,JAM,c4919d4e-af68-44cd-94ae-e716e3313269,2.0,348,JAM_HEAVY_TRAFFIC,NaN,2022-11-13 01:21:33,NaN,NaN,-43.365433,-22.979868
4,BR,Rio de Janeiro,3,0,6,JAM,0c0604cc-439c-4031-862b-703cb579effc,2.0,160,JAM_MODERATE_TRAFFIC,Vd. São Sebastião,2022-11-13 01:21:26,0.0,NaN,-43.197115,-22.908353
